In [1]:
NONE = 0b00000000
EOF = -1
SP_OR_COMM,KW,ID,OP,INT,FP,DEL,STR = 'sp_comm','k','id','op','int','fp','del','str'
YES_SP_OR_COMM, YES_KW, YES_ID, YES_OP, YES_INT, YES_FP, YES_DEL, YES_STR = 0b10000000, 0b01000000, 0b00100000, 0b00010000, 0b00001000, 0b00000100, 0b00000010, 0b00000001

In [2]:
charSet = [
    'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
    'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
    '0','1','2','3','4','5','6','7','8','9','_'
]

TokenNo = 1

keywords = ['int', 'float', 'boolean', 'string',
            'while', 'until', 'if else', 'true', 'false']
keywordEnm = list(enumerate(keywords, start=TokenNo))
TokenNo += len(keywords)

delimiters = ['{', '}', '[', ']', '(', ')', ';', ',']
delimEnm = list(enumerate(delimiters, start=TokenNo))
TokenNo += len(delimiters)

operators = ['+', '-', '*', '/', '%', '?', '>', '<', '!', ':',
            ':=', '==', '>=', '<=', '!=', '&&', '||']
opEnm = list(enumerate(operators, start=TokenNo))
TokenNo += len(operators)

In [3]:
def getNext():
    return ''

In [4]:
def util(str, fp, state):
    return [str, fp, state]

In [5]:
s = "op"
s[0:0]

''

In [6]:
def isKeyword(str):
    fp = 0
    try:
        str[fp]
        for kw in keywords:
            if str[fp:len(kw)] == kw:
                fp+=len(kw)
                return util(str[:fp], fp, YES_KW)
        return util('', fp, 0b00000000)

    except(IndexError):
        return util('', EOF, 0b00000000)
print(isKeyword('i'))

['', 0, 0]


In [7]:
# with open('test2.txt', 'r') as o:
#     d = o.read()
#     print(len(d), d[3])

In [8]:
def isDelimiter(str):
    try:
        if str[0] in delimiters:
            return util(str[0],1,YES_DEL)
        else:
            return util('',1,NONE)

    except(IndexError):
        return util('', EOF, NONE)
print(isDelimiter(''))

['', -1, 0]


In [9]:
def isOp(str):
    state = OP
    fp = 0
    try:
        while True:
            if state == OP:
                if str[fp] in operators[:6]:
                    state = '1'
                elif str[fp] == ':':
                    state = ':'
                elif str[fp] == '>':
                    state = '>'
                elif str[fp] == '<':
                    state = '<'
                elif str[fp] == '!':
                    state = '!'
                elif str[fp] == '=':
                    state = '=='
                elif str[fp] == '&':
                    state = '&&'
                elif str[fp] == '|':
                    state = '||'
                else:
                    return util('', fp, NONE)
                fp+=1
                
            elif state == '1':
                return util(str[:fp], fp, YES_OP)
            
            elif state==':' or state=='>' or state=='<' or state=='!':
                try:
                    if str[fp] == '=':
                        fp+=1
                        return util(str[:fp], fp, YES_OP)
                except(IndexError):
                    return util(str[:fp], fp, YES_OP)
            
            elif state == '==':
                if str[fp] == '=':
                    fp+=1
                    return util(str[:fp], fp, YES_OP)
                else:
                    return util('', fp, NONE)
                
            elif state == '&&':
                if str[fp] == '&':
                    fp+=1
                    return util(str[:fp], fp, YES_OP)
                else:
                    return util('', fp, NONE)
                
            elif state == '||':
                if str[fp] == '|':
                    fp+=1
                    return util(str[:fp], fp, YES_OP)
                else:
                    return util('', fp, NONE)
                
    except(IndexError):
        return util('', EOF, NONE)
print(isOp(''))

['', -1, 0]


In [10]:
def isId(str):
    state = ID
    fp = 0
    try:
        while True:
            if state == ID:
                if str[fp] in charSet[:26]:
                    state = 'final'
                    fp+=1
                else:
                    return util('', fp, NONE)
            
            elif state == 'final':
                try:
                    if str[fp] in charSet:
                        state = 'final'
                        fp+=1
                    else:
                        return util(str[:fp], fp, YES_ID)
                except(IndexError):
                    return util(str[:fp], EOF, YES_ID)
    
    except(IndexError):
        return util('', EOF, NONE)    

print(isId(''))

['', -1, 0]


In [11]:
def isInt(str):
    state = INT
    fp = 0
    try:
        while True:
            if state == INT:
                if str[fp]=='+' or str[fp]=='-':
                    state = '1'
                    fp+=1
                elif str[fp] in charSet[53:62]:
                    state = '2'
                    fp+=1
                elif str[fp] == '0':
                    state = '3'
                    fp+=1
                else:
                    return util('', fp, NONE)
            
            elif state == '1':
                if str[fp] in charSet[53:62]:
                    state = '2'
                    fp+=1
                else:
                    return util('', fp, NONE)
            
            elif state == '2':
                try:
                    if str[fp] in charSet[52:62]:
                        state = '2'
                        fp+=1
                    else:
                        return util(str[:fp], fp, YES_INT)
                except(IndexError):
                    return util(str[:fp], EOF, YES_INT)
        
            elif state == '3':
                try:
                    if str[fp] == '0':
                        state = '3'
                        fp+=1
                    elif str[fp] in charSet[53:62]:
                        state = '4'
                        fp+=1
                    else:
                        return util(str[:fp], fp, YES_INT)
                except(IndexError):
                    return util(str[:fp], fp, YES_INT)
                
            elif state == '4':
                return util('', fp, NONE)
            
    except(IndexError):
        return util('', EOF, NONE)
print(isInt(''))

['', -1, 0]


In [12]:
def isFP(str):
    state = FP
    fp = 0
    try:
        while True:
            if state == FP:
                if str[fp] == '+' or str[fp] == '-':
                    state = '1'
                    fp += 1
                elif str[fp] in charSet[53:62]:
                    state = '2'
                    fp += 1
                elif str[fp] == '0':
                    state = '6'
                    fp += 1
                else:
                    return util('', fp, NONE)

            elif state == '1':
                if str[fp] in charSet[53:62]:
                    state = '2'
                    fp += 1
                elif str[fp] == '0':
                    state = '8'
                    fp+=1
                else:
                    return util('', fp, NONE)

            elif state == '2':
                if str[fp] in charSet[52:62]:
                    state = '2'
                    fp += 1
                elif str[fp] == '.':
                    state = '3'
                    fp+=1
                else:
                    return util('', fp, NONE)

            elif state == '3':
                if str[fp] == '0':
                    state = '4'
                    fp += 1
                elif str[fp] in charSet[53:62]:
                    state = '5'
                    fp+=1
                else:
                    return util('', fp, NONE)
            
            elif state == '4':
                if str[fp] in charSet[53:62]:
                    state = '5'
                    fp+=1
                elif str[fp] == '0':
                    state = '4'
                    fp+=1
                else:
                    return util('', fp, NONE)

            elif state == '5':
                try:
                    if str[fp] in charSet[52:62]:
                        state = '5'
                        fp+=1
                    else:
                        return util(str[:fp], fp, YES_FP)
                except(IndexError):
                    return util(str[:fp], EOF, YES_FP)
        
            elif state == '6':
                if str[fp] == '.':
                    state = '7'
                    fp+=1
                else:
                    return util('', fp, NONE)
                
            elif state == '7':
                if str[fp] in charSet[52:62]:
                    state = '5'
                    fp+=1
                else:
                    return util('', fp, NONE)
                
            elif state == '8':
                if str[fp] == '.':
                    state = '3'
                    fp+=1
                else:
                    return util('', fp, NONE)
    except(IndexError):
        return util('', EOF, NONE)

print(isFP(''))

['', -1, 0]


In [13]:
def isStr(str):
    # print(str)
    state = STR
    fp = 0
    try:
        while True:
            if state == STR:
                if str[fp] == '"':
                    state = '1'
                    fp+=1
                else:
                    return util('', fp, NONE)
            
            elif state == '1':
                if str[fp] == '\\':
                    state = '3'
                    fp+=1
                elif str[fp] == '"':
                    state = '2'
                    fp+=1
                elif str[fp] not in ['\n', '\r', '\\', '"']:
                    state = '1'
                    fp+=1
                else:
                    return util('', fp, NONE)
                    
            elif state == '2':
                return util(str[:fp], fp, YES_STR)
            
            elif state == '3':
                if str[fp] in ['\\','n','r','"','t']:
                    state = '1'
                    fp+=1
                else:
                    return util('', fp, NONE)

    except(IndexError)            :
        return util('', EOF, NONE)

with open('test2.txt', 'r') as f:
    x = f.read()
    print(x)
    print(isStr(x))


    intx u:=4+2;

['', 0, 0]


In [14]:
def isSpaceOrComm(str):
    state = SP_OR_COMM
    fp = 0
    try:
        while True:
            if state == SP_OR_COMM:
                if str[fp] in [' ', '\n', '\t', '\r']:
                    state = 'sp'
                    fp+=1
                elif str[fp] == '#':
                    state = 'comm'
                    fp+=1
                else:
                    return util('', fp, NONE)
            
            elif state == 'sp':
                try:
                    if str[fp] in [' ', '\n', '\t', '\r']:
                        state = 'sp'
                        fp+=1
                    else:
                        return util('', fp, YES_SP_OR_COMM)
                except(IndexError):
                    return util('', fp, YES_SP_OR_COMM)
            
            elif state == 'comm':
                try:
                    if str[fp] != '\n':
                        state = 'comm'
                        fp+=1
                    else:
                        fp+=1
                        return util('', fp, YES_SP_OR_COMM)
                except(IndexError):
                    return util('', fp, YES_SP_OR_COMM)
                
    except(IndexError):
        return util('', EOF, NONE)
            
print(isSpaceOrComm(' '))

['', 1, 128]


In [15]:
def panic_mode(str):
    fp = 0
    while True:
        try:
            if str[fp] == ';':
                return fp+1
            else:
                fp+=1
        except(IndexError):
            return

In [16]:
def binUtil(str):
    str = bin(str)[2:]
    if len(str) < 8:
        for i in range(8-len(str)):
            str = '0' + str
    return str

In [24]:
IND_SP_COMM, IND_KW, IND_ID, IND_OP, IND_INT, IND_FP, IND_DEL, IND_STR = 0, 1, 2, 3, 4, 5, 6, 7

TokenNames = {
    IND_SP_COMM: ['Space / Comment', isSpaceOrComm],
    IND_KW: ['Keyword', isKeyword],
    IND_ID: ['Identifier', isId],
    IND_OP: ['Operator', isOp],
    IND_INT: ['Integer Literal', isInt],
    IND_FP: ['Floating Point Literal', isFP],
    IND_DEL: ['Delimiter', isDelimiter],
    IND_STR: ['String Literal', isStr]
}
def dfa():
    lp = 0
    with open('test2.txt', 'r') as f:
        codex = f.read()
        while True:
            results, result, final = [], 0b00000000, None
            for i in range(8):
                results.append(TokenNames[i][1](codex))
                result |= results[i][2]

            result = binUtil(result)
            
            if result[IND_KW]=='1' and result[IND_ID]=='1':
                if results[IND_KW][1] >= results[IND_ID][1]:
                    final = results[IND_KW]
                else:
                    final = results[IND_ID]
            
            elif result[IND_INT] == '1' and result[IND_FP] == '1' and result[IND_OP] == '1':
                final = results[IND_OP]
                
            elif result[IND_INT] == '0' and result[IND_FP] == '1' and result[IND_OP] == '1':
                final = results[IND_FP]
                
            elif result[IND_INT] == '1' and result[IND_FP] == '0' and result[IND_OP] == '1':
                final = results[IND_INT]
                
            elif result[IND_INT] == '1' and result[IND_FP] == '1' and result[IND_OP] == '0':
                final = results[IND_FP]
                
            else:
                for index in range(8):
                    if result[index] == '1':
                        final = results[index]
                        break
            
            if final==None or final[1]==EOF:
                break
            else:
                lp = final[1]
                codex = codex[lp:]
                if final[2] != 0b10000000:
                    print(final)

dfa()

['int', 3, 64]
['main', 4, 32]
['(', 1, 2]
[')', 1, 2]
['{', 1, 2]
['string', 6, 64]
['str', 3, 32]
[':=', 2, 16]
['"sgkj\\n"', 8, 1]
[';', 1, 2]
['float', 5, 64]
['x', 1, 32]
[':=', 2, 16]
['-', 1, 16]
['2.33', 4, 4]
[';', 1, 2]
['int', 3, 64]
['y', 1, 32]
[':=', 2, 16]
['2', 1, 8]
['+2', 2, 8]
[';', 1, 2]
['}', 1, 2]
